# How to stream values from your graph

There are multiple different streaming modes.

- `values`: This streaming mode streams back values of the graph. This is the **full state of the graph** after each node is called.
- `updates`: This streaming mode streams back updates to the graph. This is the **update to the state of the graph** after each node is called.
- `messages`: This streaming mode streams back messages - both complete messages (at the end of a node) as well as **tokens** for any messages generated inside a node. This mode is primarily meant for powering chat applications.


This notebook covers `streaming_mode="values"`.

In [1]:
from langgraph_sdk import get_client

In [2]:
client = get_client()

In [3]:
# There should always be a default assistant with no configuration
assistants = await client.assistants.search()
assistants = [a for a in assistants if not a['config']]
assistants

[{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
  'graph_id': 'agent',
  'config': {},
  'created_at': '2024-05-18T00:19:39.688822+00:00',
  'updated_at': '2024-05-18T00:19:39.688822+00:00',
  'metadata': {'created_by': 'system'}}]

In [4]:
assistant = assistants[0]
assistant

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'created_at': '2024-05-18T00:19:39.688822+00:00',
 'updated_at': '2024-05-18T00:19:39.688822+00:00',
 'metadata': {'created_by': 'system'}}

In [7]:
input = {"messages": [{"role": "human", "content": "whats the weather in la"}]}
thread = await client.threads.create()
async for chunk in client.runs.stream(thread['thread_id'], assistant['assistant_id'], input=input):
    print(f"Receiving new event of type: {chunk.event}...")
    print(chunk.data)
    print("\n\n")

Receiving new event of type: metadata...
{'run_id': 'f08791ce-0a3d-44e0-836c-ff62cd2e2786'}



Receiving new event of type: values...
{'messages': [{'role': 'human', 'content': 'whats the weather in la'}]}



Receiving new event of type: values...
{'messages': [{'content': 'whats the weather in la', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'faa15565-8823-4aa1-87af-e21b40526fae', 'example': False}, {'content': [{'id': 'toolu_01E5mSaZWm5rWJnCqmt63v4g', 'input': {'query': 'weather in los angeles'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': 'run-3fe1db7a-6b8d-4d83-ba07-8657190ad811', 'example': False, 'tool_calls': [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in los angeles'}, 'id': 'toolu_01E5mSaZWm5rWJnCqmt63v4g'}], 'invalid_tool_calls': []}]}



Receiving new event of type: values...
{'messages': [{'content': 'whats the

If we want to just get the final result, we can use this endpoint and just keep track of the last value we received

In [8]:
input = {"messages": [{"role": "human", "content": "whats the weather in la"}]}
thread = await client.threads.create()
final_answer = None
async for chunk in client.runs.stream(thread['thread_id'], assistant['assistant_id'], input=input):
    if chunk.event == "values":
        final_answer = chunk.data

In [9]:
final_answer

{'messages': [{'content': 'whats the weather in la',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': 'e78c2f94-d810-42fc-a399-11f6bb1b1092',
   'example': False},
  {'content': [{'id': 'toolu_01SBMoAGr4U9x3ibztm2UUom',
     'input': {'query': 'weather in los angeles'},
     'name': 'tavily_search_results_json',
     'type': 'tool_use'}],
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': 'run-80767ab8-09fc-40ec-9e45-657ddef5e0b1',
   'example': False,
   'tool_calls': [{'name': 'tavily_search_results_json',
     'args': {'query': 'weather in los angeles'},
     'id': 'toolu_01SBMoAGr4U9x3ibztm2UUom'}],
   'invalid_tool_calls': []},
  {'content': '[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'local